# Predição de Preço de Carbono
Este notebook realiza a predição do preço de carbono utilizando diferentes modelos de machine learning. O processo inclui carregamento e preparação de dados, treinamento de modelos e avaliação de resultados.

## Importação de Bibliotecas Necessárias
Nesta etapa, importamos as bibliotecas essenciais para manipulação de dados, visualização e modelagem.

In [ ]:
# Importação de bibliotecas
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor

# Configuração de estilo para gráficos
sns.set(style='whitegrid')

## Carregamento do Dataset Consolidado
Carregamos o dataset consolidado que contém informações como PIB, emissões de gases de efeito estufa (GEE) e desmatamento.

In [ ]:
# Carregar dataset consolidado
INPUT_PATHS = {"carbono_consolidado": "data/generated/carbono_serra_penitente.csv"}
df = pd.read_csv(INPUT_PATHS["carbono_consolidado"], encoding='utf-8-sig')
print("Colunas iniciais:", df.columns.tolist())

# Visualizar as primeiras linhas do dataset
display(df.head())

## Carregamento dos Preços de Carbono
Nesta etapa, carregamos os preços de carbono a partir de um arquivo Excel e transformamos os dados para o formato adequado.

In [ ]:
# Carregar preços de carbono
price_path = "data/raw/carbon-prices-latest.xlsx"
xls = pd.ExcelFile(price_path)
sheet = xls.sheet_names[0]
price_raw = pd.read_excel(xls, sheet_name=sheet, header=1)
print("Colunas do sheet de preços:", price_raw.columns.tolist())

# Visualizar as primeiras linhas do dataset de preços
display(price_raw.head())

### Transformação dos Dados de Preços
Transformamos os dados de preços para o formato "long" e filtramos apenas os dados relevantes (EU ETS).

In [ ]:
# Identificar a coluna de instrumento e colunas de anos
instrument_col = 'Instrument name'
year_cols = [c for c in price_raw.columns if isinstance(c, int)]

# Transformar preços em formato long
df_price = price_raw.melt(
    id_vars=[instrument_col],
    value_vars=year_cols,
    var_name='ano',
    value_name='carbon_price_usd'
)

# Filtrar apenas o EU ETS
df_price = df_price[df_price[instrument_col] == 'EU ETS']

# Converter tipos
df_price['ano'] = df_price['ano'].astype(int)
df_price['carbon_price_usd'] = pd.to_numeric(df_price['carbon_price_usd'], errors='coerce')

# Selecionar e limpar
df_price = df_price[['ano', 'carbon_price_usd']].dropna().drop_duplicates()
print(f"Preços disponíveis: {df_price['ano'].min()}–{df_price['ano'].max()}")

# Visualizar os dados transformados
display(df_price.head())

## Mesclagem dos Dados
Mesclamos os preços de carbono ao dataset principal com base no ano.

In [ ]:
# Mesclar preços ao dataset principal
df = df.merge(df_price, on='ano', how='inner')
print("Dataset após merge:", df.shape)

# Visualizar as primeiras linhas do dataset mesclado
display(df.head())

### Agregação de Dados por Município e Ano
Agregamos os dados para evitar duplicações, mantendo uma linha única por município e ano.

In [ ]:
# Agregar dados por município e ano
df = (
    df.groupby(['municipio', 'ano'], as_index=False)
      .agg({
          'pib': 'first',                  # PIB já é único por muni-ano
          'GEE_tCO2e': 'sum',              # soma emissões de todas as classes
          'area_desmatada_ha': 'sum',      # soma área desmatada total
          'carbon_price_usd': 'first'      # preço único por ano
      })
)
print("Após agregação município-ano:", df.shape)

# Visualizar as primeiras linhas do dataset agregado
display(df.head())

## Preparação de Features e Target
Selecionamos as colunas de features e a variável target para o modelo.

In [ ]:
# Preparar features e target
FEATURE_COLS = ['pib', 'GEE_tCO2e', 'area_desmatada_ha']
for col in FEATURE_COLS:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
target_col = 'carbon_price_usd'

# Visualizar as features e target
display(df[FEATURE_COLS + [target_col]].head())

## Divisão dos Dados em Treino e Teste
Dividimos os dados em conjuntos de treino e teste para validação do modelo.

In [ ]:
# Divisão treino/teste
X = df[FEATURE_COLS]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Visualizar tamanhos dos conjuntos
print(f"Tamanho do treino: {X_train.shape}, Tamanho do teste: {X_test.shape}")

## Padronização dos Dados
Padronizamos os dados para melhorar o desempenho dos modelos.

In [ ]:
# Padronização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Treinamento e Avaliação de Modelos
Treinamos diferentes modelos de machine learning e avaliamos seus desempenhos.

In [ ]:
# Definir e treinar modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'KNN Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'MLP Regressor': MLPRegressor(max_iter=1000, random_state=42),
    'Lasso': Lasso(alpha=0.01, random_state=42),
    'SVR': SVR(kernel='rbf'),
    'Dummy': DummyRegressor(),
    'XGBoost': XGBRegressor(random_state=42)
}

results = []
for name, model in modelos.items():
    print(f"Treinando {name}...")
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    r2 = r2_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    print(f"{name} → R²: {r2:.3f}, MSE: {mse:.3f}")
    results.append({'model': name, 'R2': r2, 'MSE': mse})

# Visualizar resultados
df_res = pd.DataFrame(results)
display(df_res)

## Visualização de Resultados
Criamos um gráfico de dispersão para comparar os valores reais e previstos pelo melhor modelo (XGBoost).

In [ ]:
# 10) Gerar scatter Real vs Previsto para cada modelo
fig_dir = os.path.dirname(OUTPUT_PATHS.scatter_xgboost_png)
for model_name, model in modelos.items():
    y_pred = model.predict(X_test_scaled)
    slug = model_name.lower().replace(' ', '_')
    scatter_path = os.path.join(fig_dir, f"scatter_real_vs_pred_{slug}.png")
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=y_test, y=y_pred)
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
    plt.xlabel('Real')
    plt.ylabel('Previsto')
    plt.title(f'Real vs Previsto – {model_name}')
    plt.tight_layout()
    plt.savefig(scatter_path)
    plt.close()
    print(f"Scatter salvo para {model_name} em: {scatter_path}")

In [ ]:
# Importação de bibliotecas necessárias
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from variaveis import FEATURE_COLS, INPUT_PATHS, OUTPUT_PATHS

# Configuração de Estilo de Plots
Configura o estilo dos gráficos para melhor visualização e cria diretórios necessários para salvar os resultados.

In [ ]:
# Configuração de plots
sns.set(style='whitegrid')
os.makedirs(os.path.dirname(OUTPUT_PATHS.scatter_xgboost_png), exist_ok=True)

# Carregar Dataset Consolidado
Carrega o dataset consolidado contendo informações como PIB, GEE e desmatamento.

In [ ]:
# 1) Carregar dataset consolidado (PIB, GEE, desmatamento)
df = pd.read_csv(INPUT_PATHS.carbono_consolidado, encoding='utf-8-sig')
print("Colunas iniciais:", df.columns.tolist())
# Visualizar as primeiras linhas do dataset
display(df.head())

# Carregar Preços de Carbono
Carrega os preços de carbono a partir de um arquivo Excel e identifica as colunas relevantes.

In [ ]:
# 2) Carregar preços de carbono
price_path = INPUT_PATHS.carbon_prices_raw
xls = pd.ExcelFile(price_path)
sheet = xls.sheet_names[0]
price_raw = pd.read_excel(xls, sheet_name=sheet, header=1)
print("Colunas do sheet de preços:", price_raw.columns.tolist())
# Visualizar as primeiras linhas do dataset de preços
display(price_raw.head())

# Transformar Preços em Formato Long
Transforma os preços de carbono em formato long para facilitar a análise e filtra apenas os dados relevantes.

In [ ]:
# Identificar a coluna de instrumento (Instrument name)
instrument_col = 'Instrument name'
# Identificar colunas de anos (tipos int)
year_cols = [c for c in price_raw.columns if isinstance(c, int)]

# 3) Transformar preços em formato long
df_price = price_raw.melt(
    id_vars=[instrument_col],
    value_vars=year_cols,
    var_name='ano',
    value_name='carbon_price_usd'
)
# Filtrar apenas o EU ETS
df_price = df_price[df_price[instrument_col] == 'EU ETS']
# Converter tipos
df_price['ano'] = df_price['ano'].astype(int)
df_price['carbon_price_usd'] = pd.to_numeric(
    df_price['carbon_price_usd'], errors='coerce')
# Selecionar e limpar
df_price = df_price[['ano', 'carbon_price_usd']].dropna().drop_duplicates()
print(f"Preços disponíveis: {df_price['ano'].min()}–{df_price['ano'].max()}")
# Visualizar os preços transformados
display(df_price.head())

# Mesclar Preços ao Dataset Principal
Mescla os preços de carbono ao dataset principal com base no ano.

In [ ]:
# 4) Mesclar preços ao dataset principal
df = df.merge(df_price, on='ano', how='inner')
print("Dataset após merge:", df.shape)
# Visualizar as primeiras linhas do dataset mesclado
display(df.head())

# Agregar Dados por Município e Ano
Agrupa os dados por município e ano para evitar duplicações e mantém uma linha única por município-ano.

In [ ]:
# 4.1) Agregar dados por município e ano para evitar duplicações
df = (
    df.groupby(['municipio', 'ano'], as_index=False)
      .agg({
          'pib': 'first',                  # PIB já é único por muni-ano
          'GEE_tCO2e': 'sum',              # soma emissões de todas as classes
          'area_desmatada_ha': 'sum',      # soma área desmatada total
          'carbon_price_usd': 'first'      # preço único por ano
      })
)
print("Após agregação município-ano:", df.shape)
# Visualizar as primeiras linhas do dataset agregado
display(df.head())

# Preparar Features e Target
Prepara as colunas de features e a variável target para o treinamento dos modelos.

In [ ]:
# 5) Preparar features e target
feature_cols = FEATURE_COLS
for col in feature_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
target_col = 'carbon_price_usd'

# Divisão Treino/Teste
Divide os dados em conjuntos de treino e teste.

In [ ]:
# 6) Divisão treino/teste
X = df[feature_cols]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Padronização
Padroniza os dados para melhorar o desempenho dos modelos.

In [ ]:
# 7) Padronização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir e Treinar Modelos
Define diferentes modelos de machine learning e realiza o treinamento.

In [ ]:
# 8) Definir e treinar modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'KNN Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'MLP Regressor': MLPRegressor(max_iter=1000, random_state=42),
    'Lasso': Lasso(alpha=0.01, random_state=42),
    'SVR': SVR(kernel='rbf'),
    'Dummy': DummyRegressor(),
    'XGBoost': XGBRegressor(random_state=42)
}

results = []
for name, model in modelos.items():
    print(f"Treinando {name}...")
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    r2 = r2_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    print(f"{name} → R²: {r2:.3f}, MSE: {mse:.3f}")
    results.append({'model': name, 'R2': r2, 'MSE': mse})

# Salvar Resultados
Salva os resultados dos modelos em um arquivo CSV.

In [ ]:
# 9) Salvar resultados
df_res = pd.DataFrame(results)
df_res.to_csv(OUTPUT_PATHS.model_results_csv, index=False)
print("Resultados salvos em:", OUTPUT_PATHS.model_results_csv)
# Visualizar os resultados
display(df_res)

# Scatter Real vs Previsto – XGBoost
Gera um gráfico de dispersão comparando os valores reais e previstos pelo modelo XGBoost.

In [ ]:
# 10) Scatter Real vs Previsto – XGBoost
best = modelos['XGBoost']
y_best = best.predict(X_test_scaled)
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test, y=y_best)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel('Real')
plt.ylabel('Previsto')
plt.title('Real vs Previsto – XGBoost')
plt.tight_layout()
plt.savefig(OUTPUT_PATHS.scatter_xgboost_png)
print("Scatter salvo em:", OUTPUT_PATHS.scatter_xgboost_png)